#### Pre-code process

##### Packages

In [7]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt

##### Read data

In [8]:
df = pd.read_csv('OR_hw02_data.csv', header=None)
print(df.to_string()) 

p = 8 # build at most p parks
n, m = df.shape
print(df.shape) # (m=60, n=20)
towns  = range(m)
potential_locations = range(n)

     0    1    2    3    4    5    6    7    8    9    10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59
0   220  216  340  266  232  281  277  233  219  233  317  301  228  238  277  223  220  272  307  245  267  240  227  215  217  226  223  240  272  242  223  269  234  217  258  227  289  300  216  328  299  323  236  215  258  273  216  310  260  216  324  233  227  230  215  230  219  221  284  269
1    88   78  274  174  115  196  190  117   87  117  245  223  107  126  189   95   87  183  231  140  175  131  105   76   81  102   97  131  182  135   96  178  119   81  161  105  207  222   78  259  221  252  123   75  161  185   77  235  165   78  254  117  105  111   76  110   85   90  201  178
2    77   66  271  169  107  191  186  109   76  109  241  219   98  119  185   85   77  17

#### Model

In [9]:
eg4d = Model("eg4d")

#-------- Add variables as a list ---------#
# xj = 1 if a park is built at loc j
x = []
for j in potential_locations:
    x.append(eg4d.addVar(lb=0, vtype = GRB.BINARY, name = "x" + str(j+1)))

# yij = 1 if the park in location j is the closest one for town i
y = []
for i in towns:
    y.append([])
    for j in potential_locations:
        y[i].append(eg4d.addVar(lb = 0, vtype = GRB.BINARY, name = "y" + str(i+1) + str(j+1)))

# dij = the distance between town i and location j
d = []
for i in towns:
    d.append([])
    for j in potential_locations:
        d[i].append(eg4d.addVar(lb = df.iloc[j, i], vtype = GRB.INTEGER, name = "d" + str(i+1) + str(j+1)))
        
# build at most p parks in potential locations.
p = eg4d.addVar(lb = 0, vtype = GRB.INTEGER, name = "p")

# w = the maximum distance for each people to move to her/his closest park.
w = eg4d.addVar(lb = 0, vtype = GRB.INTEGER, name = "max_distance")

##### Objective Function

In [10]:
eg4d.setObjective(w,GRB.MINIMIZE)

##### Constraints

In [ ]:
eg4d.addConstrs((quicksum(y[i][j] for j in potential_locations) == 1 for i in towns), "每個town都有一個最近的park")
eg4d.addConstrs((y[i][j] <= x[j] for i in towns for j in potential_locations), "要確定那個park有蓋")
eg4d.addConstr((quicksum(x[j] for j in potential_locations) <= p), "最多蓋p個park")
eg4d.addConstrs((d[i][j] * y[i][j] <= w for i in towns for j in potential_locations), "每個town到最近的park的最遠距離")
#eg4d.addConstrs((quicksum(d[i][j] * y[i][j] for j in potential_locations) <= w for i in towns), "每個town到最近的park的最遠距離")

##### Optimize

In [12]:
eg4d.optimize()
print("z* = ", eg4d.ObjVal)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1261 rows, 2422 columns and 3621 nonzeros
Model fingerprint: 0xfb78fc48
Model has 1200 quadratic constraints
Variable types: 0 continuous, 2422 integer (1220 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 1e+00]
Presolve removed 1201 rows and 21 columns
Presolve time: 0.01s
Presolved: 3660 rows, 6001 columns, 9600 nonzeros
Presolved model has 2400 SOS constraint(s)
Variable types: 0 continuous, 6001 integer (2400 binary)
Found heuristic solution: objective 456.0000000
Found heuristic solution: objective 447.0000000
Found heuristic solution: objective 340.0000000